In [1]:
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import json
import time
import pytensor 
import pymc as pm
import arviz as az
import jax
import jax.numpy as jnp
import pyhf
pyhf.set_backend('jax')

from jax import grad, jit, vmap, value_and_grad, random
from pytensor import tensor as pt
from pytensor.graph.basic import Apply
from pytensor.graph import Apply, Op

# Study on the Parameter Priors

## Op

In [60]:
### Class that creates the model Op
class Op(pt.Op):
    itypes = [pt.dvector]  # Expects a vector of parameter values
    otypes = [pt.dvector]  # Outputs a vector of values (the model.expected_actualdata)

    def __init__(self, name, func):
        ## Add inputs as class attributes
        self.func = func
        self.name = name

    def perform(self, node, inputs, outputs):
        ## Method that is used when calling the Op
        (theta,) = inputs  # Contains my variables

        ## Calling input function (in our case the model.expected_actualdata)
        result = self.func(theta)

        ## Output values of model.expected_actualdata
        outputs[0][0] = np.asarray(result, dtype=node.outputs[0].dtype)

## Dummy Example

... Testing the stitching of parateters for a simple dummy model with one unconstrained parameter `mu`, one Gaussian constraint `uncorr_bkguncrt` and two Poisson constraints `corr_bkguncrt`. 

In [63]:
### Simple pyhf model with one gaussian and one poisson constraint
model = pyhf.Model({'channels': [{'name': 'singlechannel',
   'samples': [

    {'name': 'background',
     'data': [50,50],
     'modifiers': [
         {'name': 'uncorr_bkguncrt', 'type': 'shapesys','data': [5,5]},
         {'name': 'corr_bkguncrt', 'type': 'histosys','data': {'hi_data': [55,65], 'lo_data': [45,40]}}]},
                                
    {'name': 'signal',
     'data': [5,5],
     'modifiers': [
         {'name': 'mu', 'type': 'normfactor', 'data': None}]},
                                 
    ]}]}
)

obs = model.expected_actualdata(model.config.suggested_init())

print(model.config.par_map)
print(model.config.par_order)

{'corr_bkguncrt': {'slice': slice(0, 1, None), 'paramset': <pyhf.parameters.paramsets.constrained_by_normal object at 0x28b77d790>}, 'mu': {'slice': slice(1, 2, None), 'paramset': <pyhf.parameters.paramsets.unconstrained object at 0x28bcd4700>}, 'uncorr_bkguncrt': {'slice': slice(2, 4, None), 'paramset': <pyhf.parameters.paramsets.constrained_by_poisson object at 0x28bcd4520>}}
['corr_bkguncrt', 'mu', 'uncorr_bkguncrt']


In [59]:
### Test array
mu = np.array([1.1])
corr = np.array([8, 9])
uncorr = np.array([3])

pars = np.concatenate([mu, uncorr, corr])
print(test)

### Stitching
target = np.array([2, 3, 0, 1])
stitched = pars[target.argsort()]
print(stichted)

[1.1 8.  9.  3. ]
[8.  9.  1.1 3. ]


### PyMC Implementation

In [64]:
### Applying the Op with arguments (function, name)
mainOp = Op("mainOp", jax.jit(model.expected_actualdata))

### Specs
nPars = 3
N_prior = 500
N_posterior = 500

prior = "gamma"

mu = 1
sigma = 0.1
alBet = 40

### Opening the pyMC model space
start_time = time.time()
with pm.Model() as basic_model:
    ## TensorVar input parameters
    pars = [pm.Normal("mu", mu=1, sigma=0.1)]
    pars.extend(pm.Gamma(f"dummy{idx}", alpha=20, beta=20) for idx in range(nPars))
    pars = pt.as_tensor_variable(pars)

    ## Model for the model.expected_actualdata()
    main = pm.Poisson("main", mu=mainOp(pars), observed=obs)

    ## Sampling ...
    post_data = pm.sample(N_posterior)
    prior_data = pm.sample_prior_predictive(N_prior)
    post_pred = pm.sample_posterior_predictive(post_data)

print("...............................................")
# print(f"That took {(time.time() - start_time)/60} minutes ...")
dummyTime = round(time.time() - start_time, 3)
print(f"That took {dummyTime} seconds ...")

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Slice: [mu]
>Slice: [dummy0]
>Slice: [dummy1]
>Slice: [dummy2]


Sampling 4 chains for 1_000 tune and 500 draw iterations (4_000 + 2_000 draws total) took 1 seconds.
Sampling: [dummy0, dummy1, dummy2, main, mu]
Sampling: [main]


...............................................
That took 4.04 seconds ...


## ttbar and DisplacedLeptons:

In [35]:
### Choose the right .json file
n = "DisplacedLeptons"

if n == "ttbar":
    with open("/Users/malinhorstmann/Documents/pyhf_pymc/PPC/ttbar_ljets_xsec_inclusive_pruned.json") as serialized:
        spec = json.load(serialized)
    nBins = 37

if n == "DisplacedLeptons":
    with open("/Users/malinhorstmann/Documents/pyhf_pymc/PPC/SRee_SRmm_Srem.json") as serialized:
        spec = json.load(serialized)
    nBins = 3


### Create pyhf model
workspace = pyhf.Workspace(spec)

model = workspace.model()

### Observations
obs = model.expected_data(model.config.suggested_init())